<a href="https://colab.research.google.com/github/enya-yx/LangChain-Courses/blob/main/langgraph_stream_persistency_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "langchain-google-genai" "langchain" "langchain-core" "langgraph-prebuilt" "google-generativeai" "langchain_community" "docarray" "langchain_experimental" "aiosqlite"

In [ ]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
os.environ["TAVILY_API_KEY"] = userdata.get('tavily_api_key')

# Configure the generative AI library with your API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    verbose=True
)


In [15]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

# Define a Simple Agent State;
# Agent State is accessible to all parts of the graph.
class AgentState(TypedDict):
  messages: Annotated[list[AnyMessage], operator.add]

In [16]:
tool = TavilySearchResults(max_results=2)
print(tool.name)

tavily_search_results_json


In [17]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [18]:
# Define a simple agent
class Agent:
  def __init__(self, model, checkpointer, tools ,system=""):
    self.system = system
    graph = StateGraph(AgentState)
    graph.add_node("llm",self.call_llm)
    graph.add_node("action", self.take_action)
    graph.add_conditional_edges(
        "llm",
        self.exists_action,
        {True: "action", False: END}
    )
    graph.add_edge("action", "llm")
    graph.set_entry_point("llm")
    self.graph = graph.compile(checkpointer = checkpointer)
    self.tools = {t.name: t for t in tools}
    self.model = model.bind_tools(tools)

  def exists_action(self, state: AgentState):
    result = state['messages'][-1]
    return len(result.tool_calls) > 0

  def call_llm(self, state: AgentState):
    messages = state["messages"]
    if self.system:
      messages = [SystemMessage(content=self.system)] + messages
    message = self.model.invoke(messages)
    next_tools = message.tool_calls if message.tool_calls else ["END"]
    #print(f"Next tools: {next_tools}")
    return {'messages': [message]}

  def take_action(self, state: AgentState):
    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
      print(f"Calling function: {t}")
      result = self.tools[t['name']].invoke(t['args'])
      print(f"Result from calling function: ", result)
      results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
    print("Back to the model")
    return {'messages': results}

In [23]:
prompt = """You are a smart research assistant. Use the search engine to query results. \
You are allowed to make multiple calls (either together or in sequence).\
Only look up information when you are sure of what you want. \
"""
abot = Agent(llm, memory, [tool], system=prompt)

In [ ]:
messages = [HumanMessage(content = "What is the weather in Ann Arbor, MI")]
thread = {"configurable": {"thread_id": "1"}}
i = 1
for event in abot.graph.stream({"messages": messages}, thread):
  print("step-", i)
  for v in event.values():
    print(v)
  i += 1

In [ ]:
messages2 = [HumanMessage(content="What is the weather in SF")]
for event in abot.graph.stream({"messages": messages2}, thread):
  for v in event.values():
    print(v)


In [ ]:
messages3 = [HumanMessage(content="Which one is the warmer")]
for event in abot.graph.stream({"messages": messages3}, thread):
  for v in event.values():
    print(v)

In [ ]:
# Pending: apply async stream call to print each token